In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
import xgboost as xgb

def prepare_data(df):
    """
    Prepare dataset by handling dates and categorical variables
    """
    # Create a copy to avoid modifying original data
    data = df.copy()
    
    # Fill missing values
    data.fillna(0, inplace=True)
    
    # Convert date columns to datetime and extract features
    date_cols = ['Date_Registered', 'payment_datetime', 'purchased_datetime', 
                 'released_date', 'estimated_delivery_date', 'received_date']
    
    for col in date_cols:
        if col in data.columns:
            data[col] = pd.to_datetime(data[col], errors='coerce')
            # Extract numerical features from dates
            data[f'{col}_year'] = data[col].dt.year.fillna(-1).astype(int)
            data[f'{col}_month'] = data[col].dt.month.fillna(-1).astype(int)
            data[f'{col}_day'] = data[col].dt.day.fillna(-1).astype(int)
            # Drop original date column
            data = data.drop(columns=[col])
    
    # Encode categorical variables
    categorical_cols = ['Gender', 'Is_current_loyalty_program_member', 'loyalty_tier',
                       'payment_method', 'purchase_medium', 'shipping_method',
                       'product_category']
    
    # Only encode categorical columns that exist in the dataset
    existing_cat_cols = [col for col in categorical_cols if col in data.columns]
    data = pd.get_dummies(data, columns=existing_cat_cols, drop_first=True)
    
    return data

# Load datasets
print("Loading datasets...")
train_data = pd.read_csv('train_dataset.csv')
test_data = pd.read_csv('test_dataset.csv')

print("\nTrain data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

# Prepare features
print("\nPreparing features...")
X_train_full = prepare_data(train_data)
X_test = prepare_data(test_data)

# Remove non-feature columns from training data
cols_to_drop = ['customer_experience', 'user_id', 'transaction_id', 
                'order_id', 'tracking_number']
feature_cols = [col for col in X_train_full.columns 
                if col not in cols_to_drop]

X = X_train_full[feature_cols]
y = train_data['customer_experience']

print("\nNumber of features:", len(feature_cols))

# Encode target variable
le = LabelEncoder()
y = le.fit_transform(y)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Ensure test data has same columns as training data
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Initialize XGBoost classifier with optimized parameters
print("\nTraining XGBoost model...")
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1,
    objective='multi:softprob',
    eval_metric='mlogloss',
    random_state=42,
    n_jobs=-1
)

# Train the model
print("\nFitting model...")
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Make predictions on validation set
y_pred = model.predict(X_val)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=le.classes_))

# Calculate weighted F1 score
weighted_f1 = f1_score(y_val, y_pred, average='weighted')
print(f"\nWeighted F1 Score: {weighted_f1:.4f}")

# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
})
print("\nTop 10 Most Important Features:")
print(feature_importance.sort_values('importance', ascending=False).head(10))

# Make predictions on test set
print("\nMaking predictions on test set...")
test_predictions = model.predict(X_test)
test_predictions_labels = le.inverse_transform(test_predictions)

# Create submission file
submission = pd.DataFrame({
    'id': range(len(test_predictions_labels)),
    'customer_experience': test_predictions_labels
})

# Save submission file
submission.to_csv('submission3.csv', index=False)
print("\nSubmission file created successfully!")

# Verify file contents
print("\nVerifying saved submission file:")
saved_submission = pd.read_csv('submission3.csv')
print(saved_submission.head())
print("\nShape of saved submission:", saved_submission.shape)

Loading datasets...

Train data shape: (206969, 26)
Test data shape: (137971, 25)

Preparing features...

Number of features: 66

Training XGBoost model...

Fitting model...
[0]	validation_0-mlogloss:1.08641
[1]	validation_0-mlogloss:1.07617
[2]	validation_0-mlogloss:1.06874
[3]	validation_0-mlogloss:1.06241
[4]	validation_0-mlogloss:1.05689
[5]	validation_0-mlogloss:1.05131
[6]	validation_0-mlogloss:1.04583
[7]	validation_0-mlogloss:1.04103
[8]	validation_0-mlogloss:1.03801
[9]	validation_0-mlogloss:1.03539
[10]	validation_0-mlogloss:1.03201
[11]	validation_0-mlogloss:1.02904
[12]	validation_0-mlogloss:1.02644
[13]	validation_0-mlogloss:1.02419
[14]	validation_0-mlogloss:1.02221
[15]	validation_0-mlogloss:1.02073
[16]	validation_0-mlogloss:1.01934
[17]	validation_0-mlogloss:1.01800
[18]	validation_0-mlogloss:1.01674
[19]	validation_0-mlogloss:1.01559
[20]	validation_0-mlogloss:1.01462
[21]	validation_0-mlogloss:1.01373
[22]	validation_0-mlogloss:1.01318
[23]	validation_0-mlogloss:1.01